![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/logo_MCV_W.png)
# **Simple Object Tracking by Color**

####**In this lesson we'll learn:**
1. How to use an HSV Color Filter to Create a Mask and then Track our Desired Object


In [1]:
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/bmwm4.mp4

--2023-05-27 11:55:07--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/bmwm4.mp4
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.149.102
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.149.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13944807 (13M) [video/mp4]
Saving to: ‘bmwm4.mp4’

bmwm4.mp4           100%[===================>]  13.30M  8.92MB/s    in 1.5s    

2023-05-27 11:55:09 (8.92 MB/s) - ‘bmwm4.mp4’ saved [13944807/13944807]



In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import HTML
from base64 import b64encode


def imshow(title = "Image", image = None, size = 10):
    h, w = image.shape[:2]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(title)
    plt.show()

In [3]:
# define range of color in HSV
lower = np.array([20,50,90])
upper = np.array([40,255,255])

# Create empty points array
points = []

cap = cv2.VideoCapture('bmwm4.mp4')

# Get the height and width of the frame (required to be an interger)
width = int(cap.get(3)) 
height = int(cap.get(4))

out = cv2.VideoWriter('bmwm4_output.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

ret, frame = cap.read()
Height, Width = frame.shape[:2]
frame_count = 0
radius = 0

while True:
  
    ret, frame = cap.read()
    if ret:
      hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

      # Threshold the HSV image to get only green colors
      mask = cv2.inRange(hsv_img, lower, upper)
      
      contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cv2.drawContours(frame, contours, -1, (255,255,0), thickness=2)
      # Create empty centre array to store centroid center of mass
      center =  int(Height/2), int(Width/2)

      if len(contours) > 0:
          
          # Get the largest contour and its center 
          c = max(contours, key=cv2.contourArea)
          (x, y), radius = cv2.minEnclosingCircle(c)
          M = cv2.moments(c)
          
          # Sometimes small contours of a point will cause a divison by zero error
          try:
              center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

          except:
              center = int(Height/2), int(Width/2)

          # Allow only countors that have a larger than 25 pixel radius
          if radius > 25:
              
              # Draw cirlce and leave the last center creating a trail
              cv2.circle(frame, (int(x), int(y)), int(radius),(0, 0, 255), 2)
              cv2.circle(frame, center, 5, (0, 255, 0), -1)
              
          # Log center points 
          points.append(center)
      
      # If radius large enough, we use 25 pixels
      if radius > 25:
          
          # loop over the set of tracked points
          for i in range(1, len(points)):
              try:
                  cv2.line(frame, points[i - 1], points[i], (0, 255, 0), 2)
              except:
                  pass
              
          # Make frame count zero
          frame_count = 0
              
      out.write(frame)
    else:
      break

# Release camera and close any open windows
cap.release()
out.release()

In [4]:
!ffmpeg -i /content/bmwm4_output.avi bmwm4_output.mp4 -y

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [5]:
mp4 = open('bmwm4_output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [6]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.